In [13]:
!pip install flask_ngrok
!pip install fasttext

In [14]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [15]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
import string
import nltk; nltk.download('wordnet'); nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import math

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [16]:
def process_clean(text):
  text = text.lower()
  #multi space removal
  text = re.sub(' +', ' ', text)
  #removing emails
  text = re.sub(r"\S*@\S*\s?"," ", text)
  text = re.sub(r"received from:",' ',text)
  text = re.sub(r"from:",' ',text)
  text = re.sub(r"to:",' ',text)
  text = re.sub(r"subject:",' ',text)
  text = re.sub(r"sent:",' ',text)
  text = re.sub(r"ic:",' ',text)
  text = re.sub(r"cc:",' ',text)
  text = re.sub(r"bcc:",' ',text)
  #replace hyperlinks / url
  text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',text)
  text = re.sub(r'#([^\s]+)', r'\1', text)

  #remove caller from text
  #name_surname = caller.split()
  #text = re.sub(name_surname[0],'',text)
  #text = re.sub(name_surname[1],'',text)

  #removing digits which doesn't have a a-z or underscore to retrive JOB_0-9 because it will help in classification
  text = re.sub(r"\W\d+", '', text)

  #remove mentioned characters
  text = re.sub("[:\\/'?#\"\",<>().;{|=&^%$@!}\+\-\*]",' ',text)
  text = text.replace("\\",'')

  #multi space removal
  text = re.sub(' +', ' ', text)

  #Removes unicode strings like "\u002c" and "x96" """ -------------------------after lang translation
  #text = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', text)       
  #text = re.sub(r'[^\x00-\x7f]',r'',text)

  #remove all special characters ----------------------------------- cannot perform this since it will important chars which we wil be translating to english
  #text = re.sub(r'[^A-Za-z0-9]+', ' ', text)

  #removing single character -----------------------------------after translaion
  #text = re.sub(r"\s+[a-zA-Z]\s+", " ", text)

  #remove multiple spaces / Whitespace (space, tab, newline)
  text = re.sub(r"\s+"," ", text, flags = re.I)

  #remove space from start and end
  #text = re.sub(r"^\s+", "", text)
  # Remove new line characters 
  #text = re.sub(r'\n',' ',text)

  text = text.strip()
  return text


#clean_initial =pd.DataFrame()
def post_trans_clean(text):

  #removing digits which doesn't have a a-z or underscore 
  text = re.sub(r"\W\d+", '', text)

  #remove mentioned characters
  text = re.sub("[:\\/'?#\"\",<>_\_().;{|=&^%$@!}\+\-\*]",' ',text)

  #Removes unicode strings like "\u002c" and "x96" """ -------------------------after lang translation
  text = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', text)       
  text = re.sub(r'[^\x00-\x7f]',r'',text)

  #removing single character -----------------------------------after translaion
  text = re.sub(r"\s+[a-zA-Z]\s+", " ", text)

  #remove multiple spaces / Whitespace (space, tab, newline)
  text = re.sub(r"\s+"," ", text, flags = re.I)

  return text

lemmatizer = WordNetLemmatizer()
def lemma(text):
  pos_dict = {'N': wn.NOUN, 'V': wn.VERB, 'J': wn.ADJ, 'R': wn.ADV}
  return(' '.join([lemmatizer.lemmatize(w,pos_dict.get(t, wn.NOUN)) for w,t in nltk.pos_tag(text.split())]))

In [ ]:
from flask import Flask, jsonify, render_template,request
from flask_ngrok import run_with_ngrok
import fasttext
import sys
import os

os.chdir('/content/drive/My Drive/Colab Notebooks')
sys.path.append('/content/drive/My Drive/Colab Notebooks')
model = fasttext.load_model('fasttext_main.bin')
 
app = Flask(__name__)
run_with_ngrok(app)
@app.route('/',methods=['POST'])
def predict():
    # Get the data from the POST request.
    data = request.get_json(force=True)
    # Make prediction using model loaded from disk as per the data.
    if data['Short description'] == data['Description']:
      data['Full Desc'] = data['Description']
    else:
      data['Full Desc'] = data['Short description'] + ' ' + data['Description']
    data['Full Desc'] = process_clean(data['Full Desc'])
    data['Full Desc'] = post_trans_clean(data['Full Desc'])
    data['Full Desc'] = lemma(data['Full Desc'])
    predict_request = data['Full Desc']

    #Print output on server to check the feed 
    print(predict_request)
    prediction = model.predict(predict_request)
    print(prediction)

    # Take the first value of prediction
    output = prediction[0]
    print(output)
    return jsonify((output))
    

if __name__ == '__main__':
  app.run()